In [1]:
# Import libraries

%time

import os
import boto3
import sagemaker

from sagemaker import get_execution_role

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [2]:
# initialize preconfigurations
region = boto3.Session().region_name
role = get_execution_role()

In [4]:
# Create S3 bucket

# This creates a default S3 bucket where we will upload our model.
bucket = 'sagemaker-house-regressor' #sagemaker.Session().default_bucket()

bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)
print(role)
print(region)
print(bucket)
print(bucket_path)

arn:aws:iam::933618615917:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
eu-central-1
sagemaker-house-regressor
https://s3-eu-central-1.amazonaws.com/sagemaker-house-regressor


In [7]:
'''
Install xgboost as it is needed for loading the model from joblib dump file and test it before deployment.
Please note that the XGBoost version should be same as the version with which the model was trained locally in laptop.
'''
!conda install -y -c conda-forge xgboost==0.90

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::nbclient==0.5.2=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib==3.3.4=py36h5fab9bb_0
  - conda-forge/noarch::qdarkstyle==2.8.1=pyhd8ed1ab_2
  - conda-forge/linux-64::scikit-image==0.16.2=py36hb3f55d8_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/linux-64::widgetsnbextension==3.5.1=py36h5fab9bb_4
  - conda-forge/noarch::flake8==3.8.4=py_0
  - conda-forge/noarch::ipywidgets==7.6.3=pyhd3deb0d_0
  - conda-forge/noarch::typing-extensions==3.7.4.3=0
  - conda-forge/noarch::path.py==12.5.0=0
  - conda-forge/noarch::dask==2021.2.0=pyhd8ed1ab_0
  - conda-forge/noarch::nbformat==5.1.2=pyhd8ed1ab_1
  - conda-forge/linux-64::path==15.1.2=py36h5fab9bb_0
  - conda-forge/linux-64::nbconvert==6.0.7=py36

In [8]:
model_file_name = "DEMO-local-xgboost-reg-model"

In [9]:
import joblib
import xgboost

mymodel = joblib.load(model_file_name)

In [10]:
# import test file

#import json
import numpy as np


file_name = (
    "test_point_regression_sample.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    mypayload = np.loadtxt(f, delimiter=",")
    
print(mypayload)    

mymodel.predict(mypayload)

[[3.76800e-02 8.00000e+01 1.52000e+00 0.00000e+00 4.04000e-01 7.27400e+00
  3.83000e+01 7.30900e+00 2.00000e+00 3.29000e+02 1.26000e+01 3.92200e+02
  6.62000e+00]
 [4.41780e-01 0.00000e+00 6.20000e+00 0.00000e+00 5.04000e-01 6.55200e+00
  2.14000e+01 3.37510e+00 8.00000e+00 3.07000e+02 1.74000e+01 3.80340e+02
  3.76000e+00]
 [4.52700e-02 0.00000e+00 1.19300e+01 0.00000e+00 5.73000e-01 6.12000e+00
  7.67000e+01 2.28750e+00 1.00000e+00 2.73000e+02 2.10000e+01 3.96900e+02
  9.08000e+00]
 [1.35472e+00 0.00000e+00 8.14000e+00 0.00000e+00 5.38000e-01 6.07200e+00
  1.00000e+02 4.17500e+00 4.00000e+00 3.07000e+02 2.10000e+01 3.76730e+02
  1.30400e+01]
 [2.53560e-01 0.00000e+00 9.90000e+00 0.00000e+00 5.44000e-01 5.70500e+00
  7.77000e+01 3.94500e+00 4.00000e+00 3.04000e+02 1.84000e+01 3.96420e+02
  1.15000e+01]]


array([28.420425, 24.104414, 18.68788 , 17.74451 , 18.820585],
      dtype=float32)

In [11]:
# Create a tar.gz model file as this is the format required by Sagemaker for deployment.
mymodel._Booster.save_model(model_file_name)

In [12]:
!tar czvf model.tar.gz $model_file_name

DEMO-local-xgboost-reg-model


In [13]:
# Upload the pre-trained model to S3

#### prefix in S3
prefix = "sagemaker/DEMO-xgboost-regression-byo"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

sagemaker/DEMO-xgboost-regression-byo/DEMO-local-xgboost-reg-model/model.tar.gz


In [ ]:
## Set up hosting for the model
'''
Import model into hosting
This involves creating a SageMaker model from the model file previously uploaded to S3.
'''

In [14]:
# Create a Sagemaker model
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [15]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])


https://s3-eu-central-1.amazonaws.com/sagemaker-house-regressor/sagemaker/DEMO-xgboost-regression-byo/DEMO-local-xgboost-reg-model/model.tar.gz
arn:aws:sagemaker:eu-central-1:933618615917:model/demo-local-xgboost-reg-model2022-01-11-12-39-40
CPU times: user 59.8 ms, sys: 18.9 ms, total: 78.6 ms
Wall time: 442 ms


In [16]:
# Create endpoint configuration
'''
Create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. 
In addition, the endpoint configuration describes the instance type required for model deployment.
'''
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoost-Regressor-EndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-XGBoost-Regressor-EndpointConfig-2022-01-11-12-39-44
Endpoint Config Arn: arn:aws:sagemaker:eu-central-1:933618615917:endpoint-config/demo-xgboost-regressor-endpointconfig-2022-01-11-12-39-44


In [19]:
# Create endpoint
'''
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. 
The end result is an endpoint that can be validated and incorporated into production applications.
'''
import time
endpoint_name = "DEMO-XGBoost-Regressor-Endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

DEMO-XGBoost-Regressor-Endpoint-2022-01-11-12-41-15
arn:aws:sagemaker:eu-central-1:933618615917:endpoint/demo-xgboost-regressor-endpoint-2022-01-11-12-41-15
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:eu-central-1:933618615917:endpoint/demo-xgboost-regressor-endpoint-2022-01-11-12-41-15
Status: InService


In [20]:
# Validate the model for use
'''
Now you can obtain the endpoint from the client library using the result from previous operations 
and generate classifications from the model using that endpoint.
'''
runtime_client = boto3.client("runtime.sagemaker")

In [23]:
%%time
import json


file_name = (
    "test_point_regression_sample.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    payload = f.read().strip()
    
    
print("Payload :\n")

print(payload)
print()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

##print(response)

print("Results :\n")
print()

result = response["Body"].read().decode("utf-8")

# Unpack response
print("\nPredicted Outcome Value: {}.".format(result))


Payload :

3.767999999999999822e-02,8.000000000000000000e+01,1.520000000000000018e+00,0.000000000000000000e+00,4.040000000000000258e-01,7.274000000000000021e+00,3.829999999999999716e+01,7.309000000000000163e+00,2.000000000000000000e+00,3.290000000000000000e+02,1.259999999999999964e+01,3.921999999999999886e+02,6.620000000000000107e+00
4.417800000000000060e-01,0.000000000000000000e+00,6.200000000000000178e+00,0.000000000000000000e+00,5.040000000000000036e-01,6.551999999999999602e+00,2.139999999999999858e+01,3.375100000000000211e+00,8.000000000000000000e+00,3.070000000000000000e+02,1.739999999999999858e+01,3.803399999999999750e+02,3.759999999999999787e+00
4.526999999999999774e-02,0.000000000000000000e+00,1.192999999999999972e+01,0.000000000000000000e+00,5.729999999999999538e-01,6.120000000000000107e+00,7.670000000000000284e+01,2.287500000000000089e+00,1.000000000000000000e+00,2.730000000000000000e+02,2.100000000000000000e+01,3.968999999999999773e+02,9.080000000000000071e+00
1.354719999999

In [25]:
# Delete the endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:eu-central-1:933618615917:endpoint/demo-xgboost-regressor-endpoint-2022-01-11-12-41-15".